# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [23]:
def tokenize(text):
    # Define url pattern
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in STOPWORDS]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
def build_model():
    # build NLP pipeline - count words, tf-idf, multiple output classifier
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X_train, X_test, y_train, y_test=train_test_split(X, y)
model=build_model()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
y_pred= model.predict(X_test)

In [32]:
performance=[]
for i in range(len(y_test.columns)):
    performance.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')]
                      )
performance=pd.DataFrame(performance, columns=['f1 score', 'precision', 'recall'], index=y_test.columns)
    

In [33]:
performance

,f1 score,precision,recall
related,0.811459,0.811459,0.811459
request,0.901146,0.901146,0.901146
offer,0.994194,0.994194,0.994194
aid_related,0.777998,0.777998,0.777998
medical_help,0.922231,0.922231,0.922231
medical_products,0.950344,0.950344,0.950344
search_and_rescue,0.974026,0.974026,0.974026
security,0.982277,0.982277,0.982277
military,0.967456,0.967456,0.967456
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
model.get_params()

{'memory': None,
 'steps': [('vec',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc307f2e378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
               oob_score=False, random_state=None,

In [41]:
parameters = {'clf__estimator__max_features':['sqrt', 0.5],
             'clf__estimator__n_estimators':[50, 100]}

cv = GridSearchCV(estimator=model, param_grid=parameters, cv=5, n_jobs=6)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred= cv.predict(X_test)
performance=[]
for i in range(len(y_test.columns)):
    performance.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')]
                      )
performance=pd.DataFrame(performance, columns=['f1 score', 'precision', 'recall'], index=y_test.columns)

In [ ]:
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_improved = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
                            ])
pipeline_improved.fit(X_train, y_train)
y_pred_improved = pipeline_improved.predict(X_test)

In [ ]:
y_pred= pipeline_improved.predict(X_test)
performance=[]
for i in range(len(y_test.columns)):
    performance.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                       recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')]
                      )
performance=pd.DataFrame(performance, columns=['f1 score', 'precision', 'recall'], index=y_test.columns)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))

In [ ]:
pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.